In [35]:
%autosave 0

Autosave disabled


In [36]:
import torch
import torch.nn as nn
from torch.autograd import Variable


In [37]:
import cv2
from collections import deque
import numpy as np

In [38]:
coordinates={'x1':0,'y1':0,'x2':300,'y2':300}
cnn_input_size=28
num=[]

In [39]:
#the neural network class
class neuralnet(nn.Module):
    def __init__(self,in_dim,h1,h2,out_dim):
        super(neuralnet,self).__init__()
        self.net=nn.Sequential(
          nn.Linear(in_dim,h1),
          nn.Dropout(0.5),
          nn.ReLU(),
          nn.Linear(h1,h2),
          nn.Dropout(0.5),
          nn.ReLU(),
          nn.Linear(h2,out_dim),
        )
    def forward(self,x):
        x=self.net(x)
        return x

In [40]:
#model is loaded here
model=torch.load('pretrained pytorch model/model_gpu.pt')

In [43]:
cap=cv2.VideoCapture(0)
model.eval()
while True:
    _,frame= cap.read()
    
    #preprocessing of the image
    #frame=cv2.flip(frame,1)
    mask=frame[coordinates['y1']:coordinates['y2'],coordinates['y1']:coordinates['x2']]
    kernel=np.ones((5,5),np.uint8)
    mask=cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    
    mask=cv2.erode(mask,kernel,iterations=3)
    mask=cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask=cv2.dilate(mask,kernel,iterations=3)
    _, mask= cv2.threshold(mask,150,255,cv2.THRESH_BINARY_INV)    
    small = cv2.resize(mask, (cnn_input_size, cnn_input_size), interpolation = cv2.INTER_NEAREST)
    #small=cv2.flip(small,1)
    
    
    #model and prediction
    num=torch.from_numpy(small).view((-1,28*28)).type(torch.FloatTensor).cuda()
    pred=model(num).cuda()
    _,predicted= torch.max(pred.data,1)
    text="prediction "+ str(max(predicted.data.tolist()))
    
    #rectangle
    a=cv2.rectangle(frame,(coordinates['x1'],coordinates['y1']),(coordinates['x2'],coordinates['y2']),(0,255,0),3)
    
    #text to be written
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,text,(coordinates['x1']+10,coordinates['y2']-10), font, 2,(0,255,255),2,cv2.LINE_AA)
    
    
    cv2.imshow("frame",frame)
    cv2.imshow("mask",small)
    
    
    if cv2.waitKey(1) & 0xFF== ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [142]:
cap.release()
cv2.destroyAllWindows()